In [196]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [197]:
import pandas as pd
data_tongyeong= pd.read_csv('/content/drive/MyDrive/dataset/tongyeong.csv',encoding='utf-8')
data_tongyeong.head()

,url,caption,hashtags,likes,image_url
0,https://www.instagram.com/p/CMl62-AsiyS/,#통영_한잔할래요?🎶\n통영 여행백과📘\n⠀\n철이 없었죠...\n통영을 이제서야 ...,"#통영_한잔할래요, #통영, #통영카페, #통영카페추천, #통영여행, #통영여행코스...",1.0,https://scontent-icn2-1.cdninstagram.com/v/t51...
1,https://www.instagram.com/p/DMm0DdGTClm/,통영 다찌 현지인 맛집 추천!\n\n해산물 좋아하거나 날거 좋아하는 사람은 ㄹㅇ 강...,"#대구맛집, #통영여행",1.0,https://scontent-icn2-1.cdninstagram.com/v/t51...
2,https://www.instagram.com/p/DMsOYW-p_1p/,휘바 여름 페스티발\n\n통영 드레피인 풀빌라에서 당일치기 이용권이 나와서 다녀왔어...,"#정보공유, #휘바, #드레피인풀빌라, #통영",25.0,https://scontent-icn2-1.cdninstagram.com/v/t51...
3,https://www.instagram.com/p/DMmF79DyL62/,#협찬\n\n📍통영 스마일샌드\n통영 여행에서 빼놓을 수 없는 디저트 맛집 발견✨\...,"#협찬, #통영맛집, #통영디저트, #통영카페, #통영핫플, #통영기념품, #통영먹...",53.0,https://scontent-icn2-1.cdninstagram.com/v/t51...
4,https://www.instagram.com/p/DMgdDPVTm_T/,"재료준비로 분주한 날들을 보내고 있습니다.\n\n동피랑와팡, 7월 25일 금요일 🩵...",NaN,1.0,https://scontent-icn2-1.cdninstagram.com/v/t51...


In [198]:
from collections import Counter
import re

all_words = ' '.join(data_tongyeong['caption'].astype(str)).split()

cleaned_words = [
    word for word in all_words
    if len(word) > 1 or word.startswith('#')
    and not re.fullmatch(r'[\W_]+', word)
]


word_counts = Counter(cleaned_words)


most_common_words = word_counts.most_common(10)

# 출력
for word, count in most_common_words:
  print(f'{word}: {count}')
  pass
print(f'\n총 고유 단어 수: {len(word_counts)}')


통영: 220
통영시: 184
#통영여행: 130
#통영: 116
있는: 87
경남: 85
통영의: 62
함께: 52
📍경남: 49
#통영가볼만한곳: 40

총 고유 단어 수: 11543


In [199]:
experience_words = [(word, count) for word, count in most_common_words if '축제' in word]

for word, count in experience_words:
    print(f'{word}: {count}')

In [200]:
tourist_spots = [
     '바다', '광도면', '산양읍', '비진도', '욕지면', '수국', '광도빛길',
    '도산면', '펜션', '욕지도', '도산일주로', '고양이', '용호도', '이순신공원',
    '한산면', '비진도여행',  '한산도', '한산대첩광장', '거제도', '통영해수욕장',
    '디피랑', '해저터널', '더카트인통영', '욕지도여행', '동피랑', '벽화',
     '통영중앙시장', '통영케이블카', '유람선', '해금강', '죽림4로',
    '사량도', '통영한산대첩광장','서피랑'
]

transactions=[]

for caption in data_tongyeong['caption']:
    places = [spot for spot in tourist_spots if spot in caption]

    hashtags = re.findall(r'#\w+', caption)

    # tourist_spots 단어가 해시태그에 포함된 경우만 허용
    filtered_hashtags = [tag for tag in hashtags if any(spot in tag for spot in tourist_spots)]

    transaction = places + filtered_hashtags
    transactions.append(transaction)

In [201]:
group_map = {
    '욕지도': '욕지도',
    '욕지도여행': '욕지도',
    '광도빛길': '수국',
    '광도면': '수국',
    '비진도여행':'비진도',
    '벽화':'동피랑',
    '한산대첩축제':'축제',
    '도산일주로':'도산면'

    # 추가 그룹핑 가능
}


#'숨길 거':'나타낼 거'

transactions = []

for caption in data_tongyeong['caption']:
    found_spots = [spot for spot in tourist_spots if spot in caption]
    grouped_places = [group_map.get(spot, spot) for spot in found_spots]
    # 중복 제거
    unique_places = list(dict.fromkeys(grouped_places))

    hashtags = re.findall(r'#\w+', caption)
    filtered_mapped_hashtags = []
    for tag in hashtags:
        tag_text = tag[1:]
        matched_spots = [spot for spot in tourist_spots if spot in tag_text]
        if matched_spots:
            mapped = group_map.get(matched_spots[0], matched_spots[0])
            filtered_mapped_hashtags.append(mapped)
    # 중복 제거
    unique_hashtags = list(dict.fromkeys(filtered_mapped_hashtags))

    # places 와 hashtags 합치고, 전체 중복 다시 제거 (필요하면)
    transaction = unique_places + unique_hashtags
    transaction = list(dict.fromkeys(transaction))  # 전체 중복 제거

    transactions.append(transaction)

for i, t in enumerate(transactions[:10]):
    print(f"{i+1}번째 transaction:", t)


1번째 transaction: ['바다']
2번째 transaction: []
3번째 transaction: []
4번째 transaction: ['동피랑']
5번째 transaction: ['동피랑']
6번째 transaction: ['바다', '비진도']
7번째 transaction: ['비진도', '용호도', '통영해수욕장', '사량도']
8번째 transaction: []
9번째 transaction: ['욕지도']
10번째 transaction: ['고양이', '용호도']


In [202]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import pandas as pd

# ✔️ 1. 빈 트랜잭션 제거
cleaned_transactions = [t for t in transactions if t]

# ✔️ 2. 인코딩
te = TransactionEncoder()
te_ary = te.fit(cleaned_transactions).transform(cleaned_transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)

# ✔️ 3. 자주 등장하는 아이템셋
frequent_itemsets = apriori(df, min_support=0.04, use_colnames=True)

# ✔️ 4. 연관 규칙 추출
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.1)

# ✔️ 5. 산양읍을 미륵산으로 바꾸기
rules['antecedents'] = rules['antecedents'].apply(lambda x: {item.replace('산양읍', '미륵산').replace('한산면', '한산도') for item in x})
rules['consequents'] = rules['consequents'].apply(lambda x: {item.replace('산양읍', '미륵산').replace('한산면', '한산도') for item in x})



print("\n🔗 연관 규칙 (Top 10 by Lift):")
print(rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False).head(100))



🔗 연관 규칙 (Top 10 by Lift):
    antecedents  consequents   support  confidence      lift
18     {수국, 바다}      {이순신공원}  0.042945    0.538462  4.619433
21      {이순신공원}     {수국, 바다}  0.042945    0.368421  4.619433
4         {동피랑}     {통영케이블카}  0.042945    0.250000  4.527778
3      {통영케이블카}        {동피랑}  0.042945    0.777778  4.527778
19  {이순신공원, 바다}         {수국}  0.042945    0.583333  3.395833
20         {수국}  {이순신공원, 바다}  0.042945    0.250000  3.395833
15         {수국}      {이순신공원}  0.049080    0.285714  2.451128
16      {이순신공원}         {수국}  0.049080    0.421053  2.451128
1       {이순신공원}        {동피랑}  0.049080    0.421053  2.451128
2         {동피랑}      {이순신공원}  0.049080    0.285714  2.451128
14        {한산도}         {바다}  0.042945    0.875000  1.760802
17  {수국, 이순신공원}         {바다}  0.042945    0.875000  1.760802
13         {펜션}         {바다}  0.042945    0.875000  1.760802
11      {이순신공원}         {바다}  0.073620    0.631579  1.270955
12         {바다}      {이순신공원}  0.073620    0.148148  1.2709

In [203]:
#수국 & 바다는 이순신공원과 강한 관계 (향상도 4.6배, 확률 53.85%).

#이순신공원은 수국 & 바다와도 강한 관계 (확률 36.84%).

#동피랑과 통영케이블카는 매우 강한 관계 (확률 77.78%).

#미륵산과 바다는 관계가 있지만 향상도가 낮음 (1배 이하).